In [1]:
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta


In [3]:
bbugs = pd.read_csv('./bbugs_clean.csv')

In [4]:
bbugs.columns

Index(['NO_DECLARATION', 'No_QR', 'COORD_X', 'COORD_Y', 'NBR_EXTERMIN',
       'DATE_EVENT', 'YEAR_EVENT', 'MONTH_EVENT', 'WEEK_EVENT', 'LEN_TRAIT',
       'NUM_WEEKS_TRAIT', 'NOM_QR', 'NOM_ARROND', 'DATE_DECLARATION',
       'DATE_INSP_VISPRE', 'DATE_DEBUTTRAIT', 'no_qr', 'CENTROID_X',
       'CENTROID_Y'],
      dtype='object')

In [8]:
events_ = bbugs.groupby(['YEAR_EVENT', 'WEEK_EVENT', 'No_QR'], as_index=False).agg({'NO_DECLARATION':'count', 'NBR_EXTERMIN':'sum'})

In [14]:
events_ = events_.sort_values(['No_QR', 'YEAR_EVENT', 'WEEK_EVENT'])

In [17]:
events_.head(5)

,YEAR_EVENT,WEEK_EVENT,No_QR,NO_DECLARATION,NBR_EXTERMIN
42,2011,8,01,1,1.0
239,2011,16,01,2,2.0
329,2011,20,01,1,1.0
368,2011,21,01,2,2.0
448,2011,24,01,1,1.0


In [25]:
events_['WEEK_EVENT'].unique()

array([ 8, 16, 20, 21, 24, 27, 30, 32, 33, 36, 40, 44, 60,  1,  2,  3,  6,
        9, 10, 11, 12, 14, 18, 22, 25, 35,  4,  5,  7, 28, 45, 48, 55, 15,
       50], dtype=int64)

In [ ]:
bbugs_events = bbugs[['YEAR_EVENT', 'WEEK_EVENT', 'No_QR', 'NO_DECLARATION']]

In [23]:
events_b = events_.copy()

In [38]:
events_b['EXTERMIN_WEEK-1'] = 0
events_b['EXTERMIN_WEEK-2'] = 0
events_b['EXTERMIN_WEEK-3'] = 0
events_b['EXTERMIN_MONTH-1'] = 0
events_b['NUM_WEEKS_SINCE_LAST'] = 0

In [33]:
week = 1
week_offset = 2
(week - week_offset - 1) % 54 + 1

53

In [36]:
def week_year_counter(week, year, week_offset):
    week_ = (week - week_offset - 1) % 54 + 1
    if week != (week - week_offset):
        year_ = year - 1
    else:
        year_ = year
    return(week_, year_)

In [68]:
def get_condition(week, year, events):
    return ((events['WEEK_EVENT'] == week) & (events['YEAR_EVENT'] == year ))

In [127]:
for q in events_b['No_QR'].unique():
    events_bb = events_b[events_b['No_QR'] == q]
    length = len(events_bb)
    for i, rows in events_bb.iterrows():
        #print(str((i/length)*100) + " percent complete         \r")
        week_1, year_1 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 1)
        week_2, year_2 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 2)
        week_3, year_3 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 3)
        week_4, year_4 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 4)
        print(i)
        cond1 = events_bb[get_condition(week_1, year_1, events_bb)]['NBR_EXTERMIN']
        cond2 = events_bb[get_condition(week_2, year_2, events_bb)]['NBR_EXTERMIN']
        cond3 = events_bb[get_condition(week_3, year_3, events_bb)]['NBR_EXTERMIN']
        cond4 = events_bb[get_condition(week_4, year_4, events_bb)]['NBR_EXTERMIN']
        if not cond1.empty:
            print(cond1)
            events_b['EXTERMIN_WEEK-1'].iloc[i] += cond1.values[0]
            events_b['EXTERMIN_WEEK-2'].iloc[i] += cond1.values[0]
            events_b['EXTERMIN_WEEK-3'].iloc[i] += cond1.values[0]
            events_b['EXTERMIN_MONTH-1'].iloc[i] += cond1.values[0]
        if not cond2.empty:
            events_b['EXTERMIN_WEEK-2'].iloc[i] += cond2.values[0]
            events_b['EXTERMIN_WEEK-3'].iloc[i] += cond2.values[0]
            events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond2.values[0]
        if not cond3.empty:
            events_b['EXTERMIN_WEEK-3'].iloc[i] += cond3.values[0]
            events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond3.values[0]
        if not cond4.empty:
            events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond4.values[0]
      #  events_b['EXTERMIN_WEEK-2'].iloc[i] = events_bb[get_condition(week_2, year_2, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-1'].iloc[i]
       # events_b['EXTERMIN_WEEK-3'].iloc[i] = events_bb[get_condition(week_3, year_3, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-2'].iloc[i]
       # events_b['EXTERMIN_MONTH-1'].iloc[i] = events_bb[get_condition(week_4, year_4, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-3'].iloc[i]
       # events_b['NUM_WEEKS_SINCE_LAST'].iloc[i] = 

    

42
239
329
368
448
498
573
618
656
706
751
796
927
936
961
1010
1119
1259
42    1.0
Name: NBR_EXTERMIN, dtype: float64


C:\Users\Mélisande\Anaconda3\envs\VRDI\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1308
1358
1381
1439
1564
1622
1665
329    1.0
Name: NBR_EXTERMIN, dtype: float64
1702
368    2.0
Name: NBR_EXTERMIN, dtype: float64
1787
448    1.0
Name: NBR_EXTERMIN, dtype: float64
1877
1913
1970
2086
2205
936    1.0
Name: NBR_EXTERMIN, dtype: float64
2264
961    6.0
Name: NBR_EXTERMIN, dtype: float64
2316
1010    1.0
Name: NBR_EXTERMIN, dtype: float64
2380
2437
2502
1119    4.0
Name: NBR_EXTERMIN, dtype: float64
2551
2668
1259    1.0
Name: NBR_EXTERMIN, dtype: float64
2726
1308    1.0
Name: NBR_EXTERMIN, dtype: float64
2765
1358    1.0
Name: NBR_EXTERMIN, dtype: float64
2833
2926
2980
3041
3099
1622    3.0
Name: NBR_EXTERMIN, dtype: float64
3186
3247
3329
3380
3435
3556
1970    2.0
Name: NBR_EXTERMIN, dtype: float64
3609
3658
3698
2086    2.0
Name: NBR_EXTERMIN, dtype: float64
3724
3787
3824
3854
3907
2205    9.0
Name: NBR_EXTERMIN, dtype: float64
3954
2264    12.0
Name: NBR_EXTERMIN, dtype: float64
4016
2316    6.0
Name: NBR_EXTERMIN, dtype: float64
4066
2380    1.0
Name: NBR_EXTER

10734
10886
11099
11367
11486
11535
11965
10483    1.0
Name: NBR_EXTERMIN, dtype: float64
12005
10529    5.0
Name: NBR_EXTERMIN, dtype: float64
12069
10695    2.0
Name: NBR_EXTERMIN, dtype: float64
6
44
148
193
240
369
409
449
500
574
707
797
838
866
1011
1047
1094
1121
1170
1261
44    2.0
Name: NBR_EXTERMIN, dtype: float64
1309
1565
1728
1788
449    1.0
Name: NBR_EXTERMIN, dtype: float64
1878
1914
1993
2044
2087
2115
797    1.0
Name: NBR_EXTERMIN, dtype: float64
2207
2266
2317
1011    3.0
Name: NBR_EXTERMIN, dtype: float64
2382
1047    2.0
Name: NBR_EXTERMIN, dtype: float64
2439
1094    5.0
Name: NBR_EXTERMIN, dtype: float64
2503
1121    2.0
Name: NBR_EXTERMIN, dtype: float64
2553
1170    2.0
Name: NBR_EXTERMIN, dtype: float64
2615
2670
1261    3.0
Name: NBR_EXTERMIN, dtype: float64
2767
2835
2881
2928
2982
3043
3100
3147
3188
3249
1728    7.0
Name: NBR_EXTERMIN, dtype: float64
3331
3381
3436
3477
1914    2.0
Name: NBR_EXTERMIN, dtype: float64
3558
3856
3909
2207    3.0
Name: NBR_EXTE

KeyboardInterrupt: 